# Evaluate the classifier's transferability on a tiled dataset from Bern cohorts

In [ ]:
import os

import torch
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import f1_score, accuracy_score

import Abed_utils

## Load and build model

In [ ]:
patch_size = 8
im_size = 224
path_to_backbone = './ckpts/dino_deitsmall8_pretrain.pth'
path_to_classifier = './ckpts/classifier_K19_CE_100ep_one_layer.pt'

In [ ]:
backbone = Abed_utils.get_vit(patch_size, path_to_backbone)

classifier = Abed_utils.ClassificationHead(pretrained_path=path_to_classifier)

model = nn.Sequential(backbone, classifier)
model.eval()

if Abed_utils.BERN_TILES_ROOT is None:
    raise RuntimeError('labelled Bern tiles not available on this machine, run on the dataserver')

wsis = os.listdir(Abed_utils.BERN_TILES_ROOT)